In [9]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-upda

In [10]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-12-06 00:30:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.52MB/s    in 0.2s    

2022-12-06 00:30:37 (5.52 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CrimeDB").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Read in 3 Crime Datasets

In [16]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
bo_url = "https://capstone-crime-bucket.s3.amazonaws.com/crime.csv"
spark.sparkContext.addFile(bo_url)
bo_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("crime.csv"), sep=",", header=True, inferSchema=True)
bo_df.show



<bound method DataFrame.show of DataFrame[INCIDENT_NUMBER: string, OFFENSE_CODE: int, OFFENSE_CODE_GROUP: string, OFFENSE_DESCRIPTION: string, DISTRICT: string, REPORTING_AREA: string, SHOOTING: string, OCCURRED_ON_DATE: timestamp, YEAR: int, MONTH: int, DAY_OF_WEEK: string, HOUR: int, UCR_PART: string, STREET: string, Lat: double, Long: double, Location: string]>

In [17]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

la_url = "https://capstone-crime-bucket.s3.amazonaws.com/Crimes_2012-2016.csv"
spark.sparkContext.addFile(la_url)
la_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("Crimes_2012-2016.csv"), sep=",", header=True, inferSchema=True)
la_df.show

<bound method DataFrame.show of DataFrame[_c0: int, DR.NO: int, DATE.OCC: string, TIME.OCC: int, AREA: int, AREA.NAME: string, RD: int, Crm.Cd: int, CrmCd.Desc: string, Location.1: string, lat: double, long: double, Year: int]>

In [18]:
# Read in data from S3 Buckets
from pyspark import SparkFiles


bal_url = "https://capstone-crime-bucket.s3.amazonaws.com/Baltimore_Crime_Data.csv"
spark.sparkContext.addFile(bal_url)
bal_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("Baltimore_Crime_Data.csv"), sep=",", header=True, inferSchema=True)
bal_df.show



<bound method DataFrame.show of DataFrame[CrimeDate: string, CrimeTime: string, CrimeCode: string, Location: string, Description: string, Inside/Outside: string, Weapon: string, Post: int, District: string, Neighborhood: string, Longitude: double, Latitude: double, Location 1: string, Premise: string, Total Incidents: int]>

# Boston DataFrames

In [19]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_bo_df = bo_df.select(["INCIDENT_NUMBER", "OFFENSE_CODE","OFFENSE_CODE_GROUP","OFFENSE_DESCRIPTION","DISTRICT", "REPORTING_AREA", "OCCURRED_ON_DATE", "STREET", "Lat", "Long", "Location"])
review_bo_df.show()


+---------------+------------+--------------------+--------------------+--------+--------------+-------------------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|   OCCURRED_ON_DATE|           STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+-------------------+-----------------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|2018-09-02 13:00:00|       LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|           VANDALISM|     C11|           347|2018-08-21 00:00:00|         HECLA ST|42.30682138|-71.06030035|(42.30682138, -71...|
|     I182070941|        3410|               Towed| TOWED MOTOR VEHICLE|      D4|           151

In [20]:
# Create the _table DataFrame
OFFENSE_DESCRIPTION_bo_df = bo_df.groupby("OFFENSE_DESCRIPTION").agg({"OFFENSE_DESCRIPTION":"count"}).withColumnRenamed("count(OFFENSE_DESCRIPTION)", "offense_count")
OFFENSE_DESCRIPTION_bo_df.show(10)
     

+--------------------+-------------+
| OFFENSE_DESCRIPTION|offense_count|
+--------------------+-------------+
|FIREARM/WEAPON - ...|           10|
|ANNOYING AND ACCO...|          126|
|DRUGS - POSS CLAS...|          879|
|WEAPON - OTHER - ...|          623|
|PROPERTY - STOLEN...|          452|
|LARCENY THEFT OF ...|         2265|
|DRUGS - POSS CLAS...|         2117|
|LARCENY THEFT FRO...|         9069|
|        ANIMAL ABUSE|           64|
|WEAPON - FIREARM ...|          238|
+--------------------+-------------+
only showing top 10 rows



In [24]:

# Create the _table DataFrame and drop duplicates. 
INCIDENT_NUMBER_bo_df = bo_df.select(["INCIDENT_NUMBER", "OFFENSE_DESCRIPTION"]).drop_duplicates()
INCIDENT_NUMBER_bo_df.show()
     

+---------------+--------------------+
|INCIDENT_NUMBER| OFFENSE_DESCRIPTION|
+---------------+--------------------+
|     I182070343|THREATS TO DO BOD...|
|     I182069859|BURGLARY - COMMER...|
|     I182069742| LARCENY SHOPLIFTING|
|     I182069738|  LARCENY ALL OTHERS|
|     I182069712|BURGLARY - RESIDE...|
|     I182069562|DRUGS - POSS CLAS...|
|     I182069475|DRUGS - POSS CLAS...|
|     I182069278| DEMONSTRATIONS/RIOT|
|     I182069174|         BOMB THREAT|
|     I182069165|     PROPERTY - LOST|
|     I182068894|      WARRANT ARREST|
|     I182068821| LARCENY SHOPLIFTING|
|     I182067957|WEAPON - FIREARM ...|
|     I182067867|  INVESTIGATE PERSON|
|     I182067716|M/V - LEAVING SCE...|
|     I182067679|ASSAULT SIMPLE - ...|
|     I182067526|DRUGS - POSS CLAS...|
|     I182066955|  INVESTIGATE PERSON|
|     I182066847|      WARRANT ARREST|
|     I182066791|  LARCENY ALL OTHERS|
+---------------+--------------------+
only showing top 20 rows



In [25]:

# Create the review_id_table DataFrame. 
# Convert the 'OFFENSE_DESCRIPTION' column to a date datatype with to_date("OCCURRED_ON_DATE", 'yyyy-MM-dd').alias("OCCURRED_ON_DATE")
OFFENSE_DESCRIPTION_bo_df = bo_df.select(["OFFENSE_DESCRIPTION","DISTRICT", "REPORTING_AREA","Location", to_date("OCCURRED_ON_DATE", 'yyyy-MM-dd').alias("OCCURRED_ON_DATE")]).drop_duplicates()
OFFENSE_DESCRIPTION_bo_df.show()

+--------------------+--------+--------------+--------------------+----------------+
| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|            Location|OCCURRED_ON_DATE|
+--------------------+--------+--------------+--------------------+----------------+
|M/V ACCIDENT - PR...|    null|              |(42.31138549, -71...|      2018-09-02|
|SICK/INJURED/MEDI...|      E5|           563|(42.29011783, -71...|      2018-09-01|
|M/V - LEAVING SCE...|     D14|           795|(42.34926256, -71...|      2018-08-29|
|LARCENY THEFT FRO...|      B2|           311|(42.31719004, -71...|      2018-08-21|
|SICK/INJURED/MEDI...|      C6|           177|(42.33181880, -71...|      2018-08-31|
|     PROPERTY - LOST|      B2|           287|(42.33192532, -71...|      2017-08-31|
|      VERBAL DISPUTE|      C6|           936|(42.33731824, -71...|      2018-08-30|
|ANIMAL CONTROL - ...|      E5|           735|(42.28397190, -71...|      2018-08-30|
|           VANDALISM|      B2|           180|(42.32552923, -71..